# Extraction of Argumentation Scheme Annotations
* Based on extract_information_from_JSON_appraisal function from https://github.com/linatal/UNSC_multilayer_corpus/blob/main/json2csv_merge.py# Code was adapted to the needs of this thesis with the assistance from OpenAI’s ChatGPT (GPT-4).# Final logic reviewed and verified by Valentina Tretti-Beckles.

## Requirements

### Installs

In [ ]:
!pip install alive-progress

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 3.7 MB/s eta 0:00:00
  Created wheel for grapheme: filename=grapheme-0.6.0-py3-none-any.whl size=210082 sha256=fa5785697a6ca1bcceedc6e25d9fc5d7dc30a08394a0c52e793e26f5ac472804
  Stored in directory: /root/.cache/pip/wheels/ee/3b/0b/1b865800e916d671a24028d884698674138632a83fdfad4926
Successfully built grapheme


### General Requirements

In [ ]:
from alive_progress import alive_bar
import json
import pandas as pd
from pathlib import Path
import re
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Path Extraction

In [ ]:
# Creating DF for storage of extracted annotations
columns = ['fileid',
          'sentence',
          'sentence token',
          'sentence character offset begin',
          'sentence character offset end',
          'annotated argument',
          'label',
          'argument ID',
          'argument_scheme',
          'relation',
          'argument target',
          'argument character offset begin',
          'argument character offset end',
          'related sentence'
               ]
df = pd.DataFrame(columns=columns)


In [ ]:
df

,fileid,sentence,sentence token,sentence character offset begin,sentence character offset end,annotated argument,label,argument ID,argument_scheme,relation,argument target,argument character offset begin,argument character offset end,related sentence


In [ ]:
# Setting directories
print('retrieving data: argument\n')
arguments = "arguments"
argument_directory= "/content/drive/MyDrive/Master Thesis - 2024 2025/Programming/Extracting Annotations from Inception/argumentation-scheme-in-unsc-speeches/annotation"
output_file_argument = "/content/dataset/output_argmuments.csv"

retrieving data: argument



In [ ]:
# Creating output file
# Specify the directory and file name
directory_output = '/content/dataset' # ADD directory
file_name = 'output_test_argmuments_speech.csv' # Add csv file name

# Save the CSV file in the specified directory
file_path = os.path.join(directory_output, file_name)
df.to_csv(file_path, index=False)

In [ ]:
# Function for path retrieval
# Based on extract_information_from_JSON_appraisal function from https://github.com/linatal/UNSC_multilayer_corpus/blob/main/json2csv_merge.py
def get_paths(directory, task):
    """
    Args:
        directory: directory to curation
        task: task type ("appraisal", "conflict", "arguments")

    Returns: list with all paths to CURATION_USER.json or admin.json files
    """

    _temp_file_list = []
    current_directory = Path.cwd()
    task_directory = current_directory / Path(directory)

    folders = [f for f in task_directory.iterdir() if f.is_dir()]
    #files = [f for f in task_directory.iterdir() if f.is_file()]

    for folder in folders:
        # Match name with the regex pattern
        pattern = r"(UNSC_\d+_SPV\.\d+_spch\d+)"
        match = re.search(pattern, str(folder))  # Check for a match
        if match:  # Only proceed if the match exists
            name = match.group(1)  # Extract the matched name
            # Construct paths based on the task type
            if task in ["appraisal", "conflict"]:
                filename_appr = 'CURATION_USER.json'
                file_path_appr = task_directory / Path(folder) / Path(filename_appr)
                _temp_file_list.append((name, file_path_appr))
            elif task == "arguments":
                filename_arg = 'admin.json'
                file_path_arg = task_directory / Path(folder) / Path(filename_arg)
                _temp_file_list.append((name, file_path_arg))
        else:
            print(f"No match found for folder: {folder}")  # Debugging information

    return _temp_file_list


In [ ]:
# Calling function for path retrieval
input_files_argument = get_paths(argument_directory, arguments)

In [ ]:
# Printing Paths
input_files_argument

[('UNSC_2014_SPV.7165_spch016',
  PosixPath('/content/drive/MyDrive/Master Thesis - 2024 2025/Programming/Extracting Annotations from Inception/argumentation-scheme-in-unsc-speeches/annotation/UNSC_2014_SPV.7165_spch016_sentsplit_Lithuania.txt/admin.json')),
 ('UNSC_2014_SPV.7165_spch005',
  PosixPath('/content/drive/MyDrive/Master Thesis - 2024 2025/Programming/Extracting Annotations from Inception/argumentation-scheme-in-unsc-speeches/annotation/UNSC_2014_SPV.7165_spch005_sentsplit_France.txt/admin.json')),
 ('UNSC_2014_SPV.7219_spch019',
  PosixPath('/content/drive/MyDrive/Master Thesis - 2024 2025/Programming/Extracting Annotations from Inception/argumentation-scheme-in-unsc-speeches/annotation/UNSC_2014_SPV.7219_spch019_sentsplit_Ukraine.txt/admin.json')),
 ('UNSC_2014_SPV.7154_spch017',
  PosixPath('/content/drive/MyDrive/Master Thesis - 2024 2025/Programming/Extracting Annotations from Inception/argumentation-scheme-in-unsc-speeches/annotation/UNSC_2014_SPV.7154_spch017_sentspli

## Functions for extracting Arg. Scheme Annotations
* Includes extraction of file name, speaker and speech number, and argument target.


In [ ]:
# Function for extraction information on argumentation scheme anotations
def extract_information_arg_sch_sentences_fileName(files, output_file):

# Definining columns for final df
    columns = ['fileid',
               'filename',  # New column
               'speaker', # representative country speaking in the discourse
               'speech_number',
               'sentence',
               'sentence token',
               'sentence character offset begin',
               'sentence character offset end',
               'annotated argument',
               'label',
               'argument ID',
               'argumentation scheme',
               'relation',
               'argument target',
               'argument character offset begin',
               'argument character offset end',
               'related sentences']  # New column for related sentences

    df = pd.DataFrame(columns=columns)
    error_list = []

    with alive_bar(len(files), force_tty=True) as bar:

        for document in files:
            fileid = document[0]  # Extract file ID
            filename = Path(document[1]).parts[9]  # Extract filename without extension

            # Extract speaker from filename
            match = re.search(r'_([^_]+)\.txt$', filename)
            speaker = match.group(1) if match else "Unknown"

            # Extract speech number using regex
            match = re.search(r'_spch(\d{3})', fileid)
            speech_number = match.group(1) if match else "Unknown"


            ### Open and read the JSON file ###
            with open(document[1], 'r') as file:
                data = json.load(file)

            _temp_sentences = []
            _temp_tokens = []
            _temp_text = ''
            _temp_annotations = []
            _temp_relations = {}
            _temp_new_ID = {}
            _temp_argument_targets = {}
            counter_annotation = 1


            for item in data['%FEATURE_STRUCTURES']:
                if item['%TYPE'] == 'de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Sentence':
                    _temp_sentences.append((item['begin'], item['end']))
                elif item['%TYPE'] == 'de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Token':
                    _temp_tokens.append((item['begin'], item['end']))
                elif item['%TYPE'] == 'uima.cas.Sofa':
                    _temp_text = item['sofaString']
                elif item['%TYPE'] == 'custom.Span':
                    _temp_annotations.append((item['begin'], item['end'], item['prem_claim'], item['%ID']))
                    _temp_new_ID[item['%ID']] = counter_annotation
                    counter_annotation += 1

            # Extracting relations
            for item in data['%FEATURE_STRUCTURES']:
                if item['%TYPE'] == 'custom.Relation':
                    try:
                        argumentation_scheme = item.get('argumentation_scheme', 'no scheme')
                        argument_target = item.get('Dialecticalrole', 'no target') # si esta extrayendo el target


                        # Storing argumentation schemes
                        if _temp_relations.get(_temp_new_ID[item['@Governor']]) is None:
                            _temp_relations[_temp_new_ID[item['@Governor']]] = [(_temp_new_ID[item['@Dependent']], argumentation_scheme, argument_target)]
                        else:
                            _temp_relations[_temp_new_ID[item['@Governor']]].append((_temp_new_ID[item['@Dependent']], argumentation_scheme, argument_target))

                        # Storing argument targets
                        if _temp_argument_targets.get(_temp_new_ID[item['@Governor']]) is None:
                            _temp_argument_targets[_temp_new_ID[item['@Governor']]] = [argument_target]
                        else:
                            _temp_argument_targets[_temp_new_ID[item['@Governor']]].append(argument_target) # si los esta almacenando en la lista



                    except KeyError:
                        error_list.append((fileid, item['begin'], item['end']))

            counter_token = 0
            counter_annotation = 0

            for sent in _temp_sentences:
                _temp_token_sent_list = []
                _temp_annotation_text_list = []
                _temp_annotation_label_list = []
                _temp_argument_ID_list = []
                _temp_relation_list = []
                _temp_argumentation_scheme_list = []
                _temp_annotations_begin_list = []
                _temp_annotations_end_list = []
                _temp_related_sentences = []
                _temp_argument_target_list = []

                while _temp_tokens[counter_token][1] <= sent[1] and len(_temp_tokens) - 1 != counter_token:
                    _temp_token_sent_list.append(_temp_text[_temp_tokens[counter_token][0]:_temp_tokens[counter_token][1]])
                    counter_token += 1

                if _temp_annotations:
                    _temp_annotations = sorted(_temp_annotations, key=lambda x: x[0])

                    while counter_annotation <= len(_temp_annotations) - 1 and _temp_annotations[counter_annotation][1] <= sent[1]:
                        _temp_annotation_text_list.append(_temp_text[_temp_annotations[counter_annotation][0]:_temp_annotations[counter_annotation][1]])
                        _temp_annotation_label_list.append(_temp_annotations[counter_annotation][2])
                        _temp_argument_ID_list.append(_temp_new_ID[_temp_annotations[counter_annotation][3]])
                        argument_targets = _temp_argument_targets.get(_temp_new_ID[_temp_annotations[counter_annotation][3]], [])
                        _temp_argument_target_list.append(', '.join(argument_targets) if argument_targets else 'no target')

                        relations = _temp_relations.get(_temp_new_ID[_temp_annotations[counter_annotation][3]], [])
                        _temp_relation_list.append(relations)

                        argumentation_schemes = set()
                        for relation in relations:
                            related_id = relation[0]
                            for annotation in _temp_annotations:
                                if _temp_new_ID[annotation[3]] == related_id:
                                    argumentation_schemes.add(relation[1])

                        _temp_argumentation_scheme_list.append(', '.join(argumentation_schemes) if argumentation_schemes else 'no scheme')



                        for relation in relations:
                            related_id = relation[0]
                            for annotation in _temp_annotations:
                                if _temp_new_ID[annotation[3]] == related_id:
                                    _temp_related_sentences.append(_temp_text[annotation[0]:annotation[1]])

                        _temp_annotations_begin_list.append(_temp_annotations[counter_annotation][0])
                        _temp_annotations_end_list.append(_temp_annotations[counter_annotation][1])
                        counter_annotation += 1


                new_row = pd.DataFrame([[fileid,  # fileid
                                         filename,  # filename
                                         speaker,
                                         speech_number,
                                         _temp_text[sent[0]:sent[1]],
                                         str(_temp_token_sent_list),
                                         sent[0],
                                         sent[1],
                                         _temp_annotation_text_list,
                                         _temp_annotation_label_list,
                                         _temp_argument_ID_list,
                                         _temp_argumentation_scheme_list,
                                         _temp_relation_list,
                                         _temp_argument_target_list,
                                         _temp_annotations_begin_list,
                                         _temp_annotations_end_list,
                                         _temp_related_sentences]],
                                       columns=columns)

                df = pd.concat([df, new_row], ignore_index=True)

            bar()

        df.to_csv(output_file, sep='\t', index=False)
        print(f"DataFrame printed to {output_file}")
        print('Error List: ', error_list)

    return df


In [ ]:
# Extracting argumentation schemes annotations
arg_sentences_fileName= extract_information_arg_sch_sentences_fileName(input_files_argument, output_file_argument)

on 45: DataFrame printed to /content/dataset/output_argmuments.csv
on 45: Error List:  []
|████████████████████████████████████████| 45/45 [100%] in 21.5s (2.08/s)       


In [ ]:
arg_sentences_fileName

,fileid,filename,speaker,speech_number,sentence,sentence token,sentence character offset begin,sentence character offset end,annotated argument,label,argument ID,argumentation scheme,relation,argument target,argument character offset begin,argument character offset end,related sentences
0,UNSC_2014_SPV.7165_spch016,UNSC_2014_SPV.7165_spch016_sentsplit_Lithuania...,Lithuania,016,Let me thank the Nigerian presidency for conve...,"['Let', 'me', 'thank', 'the', 'Nigerian', 'pre...",0,155,[],[],[],[],[],[],[],[],[]
1,UNSC_2014_SPV.7165_spch016,UNSC_2014_SPV.7165_spch016_sentsplit_Lithuania...,Lithuania,016,I might refer to some of the comments by our c...,"['I', 'might', 'refer', 'to', 'some', 'of', 't...",156,239,[],[],[],[],[],[],[],[],[]
2,UNSC_2014_SPV.7165_spch016,UNSC_2014_SPV.7165_spch016_sentsplit_Lithuania...,Lithuania,016,I would also like to thank Under Secretary-Gen...,"['I', 'would', 'also', 'like', 'to', 'thank', ...",240,324,[],[],[],[],[],[],[],[],[]
3,UNSC_2014_SPV.7165_spch016,UNSC_2014_SPV.7165_spch016_sentsplit_Lithuania...,Lithuania,016,"As so many of us have said here, the internati...","['As', 'so', 'many', 'of', 'us', 'have', 'said...",326,507,[],[],[],[],[],[],[],[],[]
4,UNSC_2014_SPV.7165_spch016,UNSC_2014_SPV.7165_spch016_sentsplit_Lithuania...,Lithuania,016,"But, as we know today, the Geneva statement is...","['But', ',', 'as', 'we', 'know', 'today', ',',...",508,908,[the Geneva statement is just another document...,"[claim, premise]","[2, 1]","[no scheme, evidence]","[[], [(2, evidence, Russia)]]","[no target, Russia]","[531, 615]","[614, 908]",[the Geneva statement is just another document...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1491,UNSC_2014_SPV.7154_spch012,UNSC_2014_SPV.7154_spch012_sentsplit_China.txt,China,012,We hope that the relevant parties will remain ...,"['We', 'hope', 'that', 'the', 'relevant', 'par...",168,292,[],[],[],[],[],[],[],[],[]
1492,UNSC_2014_SPV.7154_spch012,UNSC_2014_SPV.7154_spch012_sentsplit_China.txt,China,012,Settling the question of Ukraine involves the ...,"['Settling', 'the', 'question', 'of', 'Ukraine...",294,378,[Settling the question of Ukraine involves the...,[premise],[2],[positive_value],"[[(1, positive_value, Other)]]",[Other],[294],[378],[It should be considered in a balanced manner.]
1493,UNSC_2014_SPV.7154_spch012,UNSC_2014_SPV.7154_spch012_sentsplit_China.txt,China,012,It should be considered in a balanced manner.,"['It', 'should', 'be', 'considered', 'in', 'a'...",379,424,[It should be considered in a balanced manner.],[claim],[1],[no scheme],[[]],[no target],[379],[424],[]
1494,UNSC_2014_SPV.7154_spch012,UNSC_2014_SPV.7154_spch012_sentsplit_China.txt,China,012,Political and diplomatic channels should conti...,"['Political', 'and', 'diplomatic', 'channels',...",425,579,[Political and diplomatic channels should cont...,"[claim, premise]","[3, 4]","[no scheme, positive_consequence]","[[], [(3, positive_consequence, Other), (5, po...","[no target, Other, Other]","[425, 489]","[488, 579]",[Political and diplomatic channels should cont...


In [ ]:
arg_sentences_fileName.to_excel("Annotations Argumentation Schemes Ukraine.xlsx")

In [ ]:
#arg_sentences_2
# replacing with correct representative names
# "Federation" replace with "Russian Federation"
# "America" with "United States of America"
# "Ireland" wirh "United Kingdom of Great Britain and Northern Ireland"
# "Korea" with "Republic of Korea"

# Function for replacement of Representatives Names
def replace_values_in_column(df, column_name, replacements):
    df[column_name] = df[column_name].replace(replacements)
    return df

# Apply function
arg_sentences_fileName = replace_values_in_column(arg_sentences_fileName, 'speaker', replacements)


In [ ]:
arg_sentences_fileName['speaker'].value_counts()

,count
speaker,
Russian Federation,270
Ukraine,237
United States of America,181
Lithuania,140
United Kingdom of Great Britain and Northern Ireland,127
France,114
Australia,106
Luxembourg,60
Rwanda,58


In [ ]:
arg_sentences_fileName.to_excel("argument schemes annotations complete premiseToclaim.xlsx")

### Extraction of all premise-claim relations

In [ ]:
# Function for extracting all relations between premise and claims
def extract_information_premise_claim_arg_scheme(files, output_file):
    columns = ['fileid', 'filename', 'speaker', 'speechnumber', 'premise_claim_arg_scheme']
    df = pd.DataFrame(columns=columns)
    error_list = []

    with alive_bar(len(files), force_tty=True) as bar:
        for document in files:
            fileid = document[0]  # Extract file ID
            filename = Path(document[1]).parts[9]  # Extract filename without extension

            # Extract speaker from filename
            match = re.search(r'_([^_]+)\.txt$', filename)
            speaker = match.group(1) if match else "Unknown"

            # Extract speech number using regex
            match = re.search(r'_spch(\d{3})', fileid)
            speech_number = match.group(1) if match else "Unknown"

            with open(document[1], 'r') as file:
                data = json.load(file)

            _temp_text = ''
            _temp_annotations = []
            _temp_relations = {}
            _temp_new_ID = {}
            _temp_argument_targets = {}
            counter_annotation = 1

            for item in data['%FEATURE_STRUCTURES']:
                if item['%TYPE'] == 'uima.cas.Sofa':
                    _temp_text = item['sofaString']
                elif item['%TYPE'] == 'custom.Span':
                    _temp_annotations.append((item['begin'], item['end'], item['prem_claim'], item['%ID']))
                    _temp_new_ID[item['%ID']] = counter_annotation
                    counter_annotation += 1

            for item in data['%FEATURE_STRUCTURES']:
                if item['%TYPE'] == 'custom.Relation':
                    try:
                        argumentation_scheme = item.get('argumentation_scheme', 'no scheme')
                        argument_target = item.get('Dialecticalrole', 'no target') # si extrae
                       # print("argument target: ", argument_target)

                        if _temp_argument_targets.get(_temp_new_ID[item['@Governor']]) is None:
                            _temp_argument_targets[_temp_new_ID[item['@Governor']]] = [argument_target]
                        else:
                            _temp_argument_targets[_temp_new_ID[item['@Governor']]].append(argument_target) # si los esta almacenando en la lista


                        if _temp_relations.get(_temp_new_ID[item['@Governor']]) is None:
                            _temp_relations[_temp_new_ID[item['@Governor']]] = [(_temp_new_ID[item['@Dependent']], argumentation_scheme, argument_target)] #argument_target
                        else:
                            _temp_relations[_temp_new_ID[item['@Governor']]].append((_temp_new_ID[item['@Dependent']], argumentation_scheme, argument_target))#argument_target

                       # print("_temp_relations: ", _temp_relations)

                    except KeyError:
                        error_list.append((document[0], item['begin'], item['end']))

            premise_claim_arg_scheme_list = []

            for annotation in _temp_annotations:
                ann_id = _temp_new_ID[annotation[3]]
                if annotation[2] == 'premise' and ann_id in _temp_relations:
                    for related_id, scheme, target in _temp_relations[ann_id]:
                        for related_annotation in _temp_annotations:
                            if _temp_new_ID[related_annotation[3]] == related_id and related_annotation[2] == 'claim':
                                premise_claim_arg_scheme_list.append([
                                    [_temp_text[annotation[0]:annotation[1]], 'premise'],
                                    [_temp_text[related_annotation[0]:related_annotation[1]], 'claim'],
                                    scheme,
                                    target
                                ])
            # fileid, filename, speaker, speechnumber, premise_claim_arg_scheme_list
            # 'fileid', 'filename', 'speaker', 'speechnumber' 'premise_claim_arg_scheme'
            new_row = pd.DataFrame([[document[0], filename, speaker, speech_number, premise_claim_arg_scheme_list]], columns=columns)
            df = pd.concat([df, new_row], ignore_index=True)
            bar()

        df.to_csv(output_file, sep='\t', index=False)
        print(f"DataFrame saved to {output_file}")
        print('Error List:', error_list)

    return df


In [ ]:
# Calling function for extraction of claim-premise relations
arg_sentences_2 = extract_information_premise_claim_arg_scheme(input_files_argument, output_file_argument)

on 45: DataFrame saved to /content/dataset/output_argmuments.csv
on 45: Error List: []
|████████████████████████████████████████| 45/45 [100%] in 0.5s (95.26/s)       


In [ ]:
arg_sentences_2.columns

Index(['fileid', 'filename', 'speaker', 'speechnumber',
       'premise_claim_arg_scheme'],
      dtype='object')

In [ ]:
arg_sentences_2['premise_claim_arg_scheme'].iloc[0]

[[["adding to the growing list of breaches of Russia's international commitments, such as the Helsinki Final Act, the 1991 Almaty Declaration, the Budapest Memorandum of 1994, the Treaty of Friendship and Cooperation between Russia and Ukraine of l997 and the Charter of the United Nations itself.",
   'premise'],
  ['the Geneva statement is just another document to be signed and discarded by Russia,',
   'claim'],
  'evidence',
  'Russia'],
 [['Barricades in Maidan Square are being dismantled.', 'premise'],
  ['Despite growing provocations, the Ukrainian authorities have taken a number of concrete steps to follow up on the Geneva statement.',
   'claim'],
  'evidence',
  'Ukraine'],
 [['I recall my French colleague referring a number of times to the Internet and Google.',
   'premise'],
  ['Despite growing provocations, the Ukrainian authorities have taken a number of concrete steps to follow up on the Geneva statement.',
   'claim'],
  'evidence',
  'Ukraine'],
 [['Users of those will

In [ ]:
# Defining representative names for replacements
replacements = {'UN': 'United Nations','Federation': 'Russian Federation', 'America': 'United States of America', 'Korea': 'Republic of Korea', 'Ireland':'United Kingdom of Great Britain and Northern Ireland'}

# Apply function
arg_sentences_2 = replace_values_in_column(arg_sentences_2, 'speaker', replacements)


In [ ]:
arg_sentences_2

,fileid,filename,speaker,speechnumber,premise_claim_arg_scheme
0,UNSC_2014_SPV.7165_spch016,UNSC_2014_SPV.7165_spch016_sentsplit_Lithuania...,Lithuania,016,[[[adding to the growing list of breaches of R...
1,UNSC_2014_SPV.7165_spch005,UNSC_2014_SPV.7165_spch005_sentsplit_France.txt,France,005,"[[[In several eastern cities, armed militants,..."
2,UNSC_2014_SPV.7219_spch019,UNSC_2014_SPV.7219_spch019_sentsplit_Ukraine.txt,Ukraine,019,[[[The Ukrainian authorities have already call...
3,UNSC_2014_SPV.7154_spch017,UNSC_2014_SPV.7154_spch017_sentsplit_Chile.txt,Chile,017,[[[in accordance with General Assembly resolut...
4,UNSC_2014_SPV.7165_spch007,UNSC_2014_SPV.7165_spch007_sentsplit_Argentina...,Argentina,007,"[[[What we saw as willingness to agree, the wh..."
5,UNSC_2014_SPV.7219_spch008,UNSC_2014_SPV.7219_spch008_sentsplit_China.txt,China,008,[[[Our priority now is to establish the facts....
6,UNSC_2014_SPV.7219_spch025,UNSC_2014_SPV.7219_spch025_sentsplit_Indonesia...,Indonesia,025,"[[[According to various sources and reports, t..."
7,UNSC_2014_SPV.7219_spch004,UNSC_2014_SPV.7219_spch004_sentsplit_United_Ki...,United Kingdom of Great Britain and Northern I...,004,[[[Malaysia Airlines passenger flight MH-l7 ap...
8,UNSC_2014_SPV.7165_spch004,UNSC_2014_SPV.7165_spch004_sentsplit_United_Ki...,United Kingdom of Great Britain and Northern I...,004,[[[we are deeply concerned by the deterioratin...
9,UNSC_2014_SPV.7219_spch009,UNSC_2014_SPV.7219_spch009_sentsplit_Australia...,Australia,009,"[[[If, as appears increasingly likely, Flight ..."


In [ ]:
# Saving Argumentation Scheme annotations with Claim-Premise relations
arg_sentences_2.to_excel("all speeches premiseToClaim.xlsx")

In [ ]:
arg_sentences_2.columns

Index(['fileid', 'filename', 'speaker', 'speechnumber',
       'premise_claim_arg_scheme'],
      dtype='object')

### Function for dataframe creation, including:
* fileid, filename, speaker, speech number, premise, claim, argumentation scheme label and argument target

In [ ]:
def process_argumentation_data(df, column_name):
    structured_data = []

    for index, data in df.iterrows():
        fileid = data['fileid']
        filename = data['filename']
        speaker = data['speaker']
        speech_number = data['speechnumber']
        for entry in data[column_name]:
            premise = next((item[0] for item in entry if isinstance(item, list) and item[1] == 'premise'), None)
            claim = next((item[0] for item in entry if isinstance(item, list) and item[1] == 'claim'), None)
            argumentation_scheme = entry[-2] if isinstance(entry[-2], str) else None
            argument_target = entry[-1] if isinstance(entry[-1], str) else None

            structured_data.append({
                'fileid': fileid,
                'filename': filename,
                'speaker': speaker,
                'speech': speech_number,
                'premise': premise,
                'claim': claim,
                'argumentation scheme': argumentation_scheme,
                'argument target': argument_target
            })

    return pd.DataFrame(structured_data)

In [ ]:
arg_sentences_2 = process_argumentation_data(arg_sentences_2, 'premise_claim_arg_scheme')

In [ ]:
# Defining replacements
replacements_2 = {'Russia': 'Russian Federation'}

# Apply function
arg_sentences_2 = replace_values_in_column(arg_sentences_2, 'argument target', replacements_2)

In [ ]:
# Saving resulting file
arg_sentences_2.to_excel("Complete Argument Schemes Annotations Extraction with premise-claim relations.xlsx")

In [ ]:
# Saving arguments with labels
arg_sentences_2.to_excel("All arguments with labels.xlsx")